In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_csv("propertyTrans.csv")
df.shape

In [ ]:
df.columns

In [ ]:
df.drop(["Unnamed: 0", "id", "person", "daytimePhone", "foreignAddress", "firstName", "middleName", "lastName", "addressLine1", "country", "email", "stateOrProvince"], axis=1, inplace = True)

In [ ]:
df.head()

In [ ]:
df = df[df["tier1Cde"] == "RESID"]
df.shape

In [ ]:
df["tier2Cde"].value_counts()

In [ ]:
df = df[df["tier2Cde"] == "SINGLEFAM"]

df.shape

In [ ]:
df["county"].value_counts()

In [ ]:
"""
Anoka - 2
Hennepin - 27
Ramsey - 62
Washington - 82
Dakota - 19
Scott - 70
Carver - 10
"""

counties = [2, 27, 62, 82, 19, 70, 10]

metroArea = df[df.countyCde.isin(counties)]

In [ ]:
metroArea.describe()

In [ ]:
metroArea.head()

In [ ]:
condensedMA = metroArea[["totPurchaseAmt", "street1", "zip"]]
condensedMA["property"] = condensedMA["street1"] +" "+condensedMA["zip"]
condensedMA.head()

In [ ]:
condensedMA.to_csv("metroAreaCondensed.csv")

In [ ]:
import folium
from folium import plugins
import requests

jsonData = requests.get("https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/mn_minnesota_zip_codes_geo.min.json")
mnArea = jsonData.json()


In [ ]:
numListings = condensedMA.groupby('zip').count()

In [ ]:
numListings

In [ ]:
numListingsByZip = pd.DataFrame()

numListingsByZip["zipcode"] = [str(i) for i in numListings.index]
numListingsByZip["numListings"] = numListings["totPurchaseAmt"].values

numListingsByZip

In [ ]:
numListingsByZip.describe()

In [ ]:
mnMap = folium.Map(location=[44.9778, -93.2650], tiles="Stamen Toner", zoom_start=10)

mnMap.choropleth(geo_data=mnArea, data=numListingsByZip, columns=["zipcode", "numListings"], key_on='feature.properties.ZCTA5CE10', fill_color='YlGn', fill_opacity=1)

In [ ]:
mnMap